In [10]:
from __future__ import print_function
%matplotlib inline
import fluidsim

In [11]:
from fluidsim.solvers.ns2d.solver import Simul

In [12]:
Simul.create_default_params()


<params NEW_DIR_RESULTS="True" ONLY_COARSE_OPER="False" beta="0.0" nu_2="0.0"
        nu_4="0.0" nu_8="0.0" nu_m4="0.0" short_name_type_run="">
  <oper Lx="8" Ly="8" NO_SHEAR_MODES="False"
        coef_dealiasing="0.6666666666666666" nx="48" ny="48"
        type_fft="default"/>  

  <time_stepping USE_CFL="True" USE_T_END="True" cfl_coef="None" deltat0="0.2"
                 deltat_max="0.2" it_end="10" max_elapsed="None" t_end="10.0"
                 type_time_scheme="RK4"/>  

  <init_fields available_types="['from_file', 'from_simul', 'in_script',
               'constant', 'noise', 'jet', 'dipole']" modif_after_init="False"
               type="constant">
    <from_file path=""/>  

    <constant value="1.0"/>  

    <noise length="0.0" velo_max="1.0"/>  

  </init_fields>

  <forcing available_types="['in_script', 'in_script_coarse', 'proportional',
           'tcrandom', 'tcrandom_anisotropic']" enable="False"
           forcing_rate="1.0" key_forced="None" nkmax_forcing="5"
   

## Create new  Noise initializer to remove constant seed so we can override

In [13]:
import numpy as np

from fluiddyn.util import mpi

from fluidsim.base.init_fields import InitFieldsBase, SpecificInitFields

class InitFieldsNoise2(SpecificInitFields):
    """Initialize the state with noise."""

    tag = "noise"

    @classmethod
    def _complete_params_with_default(cls, params):
        """Complete the `params` container (class method)."""
        super()._complete_params_with_default(params)

        params.init_fields._set_child(
            cls.tag, attribs={"velo_max": 1.0, "length": 0.0}
        )

        params.init_fields.noise._set_doc(
            """
velo_max: float (default 1.)

    Maximum velocity.

length: float (default 0.)

    The smallest (cutoff) scale in the noise.

"""
        )

    def __call__(self):
        rot_fft, ux_fft, uy_fft = self.compute_rotuxuy_fft()
        # energy_fft = 0.5 * (np.abs(ux_fft) ** 2 + np.abs(uy_fft) ** 2)
        self.sim.state.init_from_rotfft(rot_fft)

    def compute_rotuxuy_fft(self):

        params = self.sim.params
        oper = self.sim.oper

        lambda0 = params.init_fields.noise.length
        if lambda0 == 0:
            lambda0 = min(oper.Lx, oper.Ly) / 4.0

        def H_smooth(x, delta):
            return (1.0 + np.tanh(2 * np.pi * x / delta)) / 2.0

        ux_fft = (
            np.random.random(oper.shapeK)
            + 1j * np.random.random(oper.shapeK)
            - 0.5
            - 0.5j
        )
        uy_fft = (
            np.random.random(oper.shapeK)
            + 1j * np.random.random(oper.shapeK)
            - 0.5
            - 0.5j
        )

        if mpi.rank == 0:
            ux_fft[0, 0] = 0.0
            uy_fft[0, 0] = 0.0

        oper.projection_perp(ux_fft, uy_fft)
        oper.dealiasing(ux_fft, uy_fft)

        k0 = 2 * np.pi / lambda0
        delta_k0 = 1.0 * k0
        ux_fft = ux_fft * H_smooth(k0 - oper.K, delta_k0)
        uy_fft = uy_fft * H_smooth(k0 - oper.K, delta_k0)

        ux = oper.ifft2(ux_fft)
        uy = oper.ifft2(uy_fft)
        velo_max = np.sqrt(ux ** 2 + uy ** 2).max()
        if mpi.nb_proc > 1:
            velo_max = oper.comm.allreduce(velo_max, op=mpi.MPI.MAX)
        ux = params.init_fields.noise.velo_max * ux / velo_max
        uy = params.init_fields.noise.velo_max * uy / velo_max

        ux_fft = oper.fft2(ux)
        uy_fft = oper.fft2(uy)

        rot_fft = oper.rotfft_from_vecfft(ux_fft, uy_fft)
        return rot_fft, ux_fft, uy_fft

In [14]:
# override InitFieldsNoise with our initializer that allows changed random seeds
fluidsim.solvers.ns2d.init_fields.InitFieldsNoise = InitFieldsNoise2

## Run Simulations and save

In [1]:
import fluiddyn as fld
from fluidsim.solvers.ns2d.solver import Simul

params = Simul.create_default_params()

params.short_name_type_run = 'test'

params.oper.nx = params.oper.ny = nh = 32*2
params.oper.Lx = params.oper.Ly = Lh = 10.

delta_x = Lh / nh
params.nu_8 = 2e-3*params.forcing.forcing_rate**(1./3)*delta_x**8

params.time_stepping.t_end = 10.

params.init_fields.type = 'noise'

params.forcing.enable = True
params.forcing.type = 'tcrandom'

params.output.sub_directory = 'training_data_test'

params.output.periods_plot.phys_fields = 0.1
params.output.periods_save.phys_fields = 0.2
params.output.periods_save.spatial_means = 0.05

params.output.ONLINE_PLOT_OK = True

for i in range(10):

    sim = Simul(params)
    # InitFieldsNoise2(sim)
    sim.time_stepping.start()

    print("""
    A movie can be produced with the command (using ffmpeg):

    sim.output.phys_fields.animate(dt_frame_in_sec=0.1, dt_equations=0.08, repeat=False, save_file=1, tmax=10)
    """)

    fld.show()

C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\operators\operators2d.py:27: UserWarning: operators2d.py has to be pythranized to be efficient! Install pythran and recompile.
  "operators2d.py has to be pythranized to be efficient! "


*************************************
Program fluidsim
To plot the forcing modes, you can use:
sim.forcing.forcing_maker.plot_forcing_region()
sim:                <class 'fluidsim.solvers.ns2d.solver.Simul'>
sim.oper:           <class 'fluidsim.operators.operators2d.OperatorsPseudoSpectral2D'>
sim.output:         <class 'fluidsim.solvers.ns2d.output.Output'>
sim.state:          <class 'fluidsim.solvers.ns2d.state.StateNS2D'>
sim.time_stepping:  <class 'fluidsim.base.time_stepping.pseudo_spect.TimeSteppingPseudoSpectral'>
sim.init_fields:    <class 'fluidsim.solvers.ns2d.init_fields.InitFieldsNS2D'>
sim.forcing:        <class 'fluidsim.solvers.ns2d.forcing.ForcingNS2D'>

solver NS2D, RK4 and sequential,
type fft: fluidfft.fft2d.with_pyfftw
nx =     64 ; ny =     64
lx = 10 ; ly = 10
path_run =
C:\Users\sbendl\Sim_data\training_data_test\NS2D_test_64x64_S10x10_2020-03-10_20-25-22
init_fields.type: noise



C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\base\time_stepping\base.py:108: UserWarning: Cannot handle signals - is multithreading on?
  warn("Cannot handle signals - is multithreading on?")


Initialization outputs:
sim.output.phys_fields:       <class 'fluidsim.base.output.phys_fields2d.PhysFieldsBase2D'>


<Figure size 640x480 with 2 Axes>

...

sim.output.spectra:           <class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'>
sim.output.spectra_multidim:  <class 'fluidsim.solvers.ns2d.output.spectra_multidim.SpectraMultiDimNS2D'>
sim.output.increments:        <class 'fluidsim.base.output.increments.Increments'>
sim.output.spatial_means:     <class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'>
sim.output.spect_energy_budg: <class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'>

Memory usage at the end of init. (equiv. seq.): 157.52734375 Mo
Size of state_spect (equiv. seq.): 0.033791999999999996 Mo
*************************************
Beginning of the computation
save state_phys in file state_phys_t000.000.nc
    compute until t =         10
it =      0 ; t =          0 ; deltat  =   0.087226
              energy = 8.622e-02 ; Delta energy = +0.000e+00
MEMORY_USAGE:                  157.53515625 Mo
save state_phys in file state_phys_t000.258.nc
save state_phys in file state_

<Figure size 640x480 with 2 Axes>

sim.output.spectra:           <class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'>
sim.output.spectra_multidim:  <class 'fluidsim.solvers.ns2d.output.spectra_multidim.SpectraMultiDimNS2D'>
sim.output.increments:        <class 'fluidsim.base.output.increments.Increments'>
sim.output.spatial_means:     <class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'>
sim.output.spect_energy_budg: <class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'>

Memory usage at the end of init. (equiv. seq.): 157.828125 Mo
Size of state_spect (equiv. seq.): 0.033791999999999996 Mo
*************************************
Beginning of the computation
save state_phys in file state_phys_t000.000.nc
    compute until t =         10
it =      0 ; t =          0 ; deltat  =   0.087226
              energy = 8.622e-02 ; Delta energy = +0.000e+00
MEMORY_USAGE:                  158.046875 Mo
save state_phys in file state_phys_t000.258.nc
save state_phys in file state_phys

<Figure size 640x480 with 2 Axes>

sim.output.spectra:           <class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'>
sim.output.spectra_multidim:  <class 'fluidsim.solvers.ns2d.output.spectra_multidim.SpectraMultiDimNS2D'>
sim.output.increments:        <class 'fluidsim.base.output.increments.Increments'>
sim.output.spatial_means:     <class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'>
sim.output.spect_energy_budg: <class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'>

Memory usage at the end of init. (equiv. seq.): 158.10546875 Mo
Size of state_spect (equiv. seq.): 0.033791999999999996 Mo
*************************************
Beginning of the computation
save state_phys in file state_phys_t000.000.nc
    compute until t =         10
it =      0 ; t =          0 ; deltat  =   0.087226
              energy = 8.622e-02 ; Delta energy = +0.000e+00
MEMORY_USAGE:                  158.2265625 Mo
save state_phys in file state_phys_t000.258.nc
save state_phys in file state_p

In [ ]:
sim.output.path_run